# 파이프라인 생성 및 스케줄링

## Overview

Jupyter notebook 으로 작업한 내용을 github 에 push 한 뒤, github 의 notebook url 과 NESOperator 를 이용하여 airflow dag 를 만들고 scheduling 합니다.

### Airflow 와 Dag repository 구성

Airflow 는 개발계(https://airflow-gke-dev.shinsegae.ai)와 운영계(https://airflow-gke.shinsegae.ai)로 이루어져 있습니다.

Github 의 각 dag repository 의 branch 는 다음과 같이 airflow 와 연결되어 있습니다.

develop branch -> 개발계

main branch -> 운영계

기존 이용중인 dag repository 가 없다면 Data&Infra담당 담당자(이형문, aiweaver@emart.com) 또는 AI/ML담당 담당자(정유선, jerryjung@emart.com)에게 repository 생성 요청해 주십시오.

## Dag file 작성 및 push

```{tip}
Dag file 작업은 data 에 접근할 필요가 없으므로 local 에서도 작업 가능합니다.
```

Dag file 저장을 위한 github repo 를 clone 받고 develop branch 로 switch
```bash
git clone https://github.com/emartdt/aiml-dag-mlops.git
git checkout -f develop
```

아래 파일을 rec_modeling_tmbr.py 로 저장 후

```python
from airflow import DAG
from datetime import datetime, timedelta
from airflow.macros.slack_plugin import get_fail_alert
from airflow.operators.dummy import DummyOperator
from airflow.operators.papermill_plugin import NesOperator

default_args = {
    'owner': 'your_id',
    'depends_on_past': False,
    'start_date': datetime(2020, 9, 15),
    'retries': 3,
    'retry_delay': timedelta(minutes=30),
    'on_failure_callback': get_fail_alert(),
}

dag_id = "rec_modeling_tmbr"

with DAG(
    dag_id,
    default_args=default_args,
    catchup=False,
    schedule_interval='0 1 * * *' # cron 스케줄링과 동일합니다. 분 시 일 월 요일 입니다. KST 기준입니다.
) as dag:
    start = DummyOperator(task_id='start')
    end = DummyOperator(task_id='end')
```

github의 dag repository 에 push
```bash
git add rec_modeling_tmbr.py
git commit -m "Add dag for tmber modeling"
git push
```

5분 정도 기다린 후 airflow 개발계 web ui 에서 dag 생성 확인

## NESOperator 로 dag 구성

아래 파일과 같이 작업한 각 jupyter notebook 을 NESOperator 를 이용하여 dag 구성을 합니다.

```python
from airflow import DAG
from datetime import datetime, timedelta
from airflow.macros.slack_plugin import get_fail_alert
from airflow.operators.dummy import DummyOperator
from airflow.operators.papermill_plugin import NesOperator

default_args = {
    'owner': 'your_id',
    'depends_on_past': False,
    'start_date': datetime(2020, 9, 15),
    'retries': 3,
    'retry_delay': timedelta(minutes=30),
    'on_failure_callback': get_fail_alert(),
}

dag_id = "rec_modeling_tmbr"
pre_processing_url = "https://github.com/emartdt/aiml-dag-mlops/..."
training_url = "https://github.com/emartdt/aiml-dag-mlops/..."
prediction_url = "https://github.com/emartdt/aiml-dag-mlops/..."
post_processing_url = "https://github.com/emartdt/aiml-dag-mlops/..."


with DAG(
    dag_id,
    default_args=default_args,
    catchup=False,
    schedule_interval='0 1 * * *'
) as dag:
    start = DummyOperator(task_id='start')
    end = DummyOperator(task_id='end')
    pre_processing = NesOperator(
        task_id="pre_processing",
        input_nb=pre_processing_url,
        parameters={
            'dt': '{{ ds }}',
        },
    )
    training = NesOperator(
        task_id="training",
        input_nb=training_url,
        parameters={
            'dt': '{{ ds }}',
        },
    )
    prediction = NesOperator(
        task_id="prediction",
        input_nb=prediction_url,
        parameters={
            'dt': '{{ ds }}',
        },
    )
    post_processing = NesOperator(
        task_id="post_processing",
        input_nb=post_processing_url,
        parameters={
            'dt': '{{ ds }}',
        },
    )
    start >> pre_processing >> training >> prediction >> post_processing >> end
```

NesOperator 의 argument 는 다음과 같습니다.
- task_id: airflow web-ui 에 표시될 task 이름
- input_nb: 노트북의 github url 주소
- parameters: 노트북 실행시 전달 될 인자값. dictionary 의 각 key/value 들이 노트북의 두번째 cell 에 key = "value" 형태로 삽입됩니다.

github의 dag repository 에 push
```bash
git add rec_modeling_tmbr.py
git commit -m "Add dag for tmber modeling"
git push
```

airflow web-ui 에서 dag 를 refresh 하면 dag 의 구성이 변경된 것을 바로 확인할 수 있습니다.

## 테스트 및 운영 반영

https://airflow-gke-dev.shinsegae.ai 에서 테스트 후 문제 없으면 develop branch 에서 main branch 로 pull request 를 하고 관련 사람들의 review 를 거친 후, main branch 로 merge 합니다. main branch 로 merge 된 후 약 5분이 지나면 https://airflow-gke.shinsegae.ai 에서 새롭게 추가 된 dag 를 확인할 수 있습니다. 정상적으로 dag 가 생성되었는지 확인 후 dag 를 on 시키면 운영계 반영이 끝나게 됩니다